# Getting Started

Rossmann은 독일을 중심으로 유럽지역 전역에 매장이 있는 대형 드럭스토어 체인입니다. 4000개 이상의 매장이 있고, 유럽 시장 업계 1,2위를 다투는 규모입니다. 이 데이터는 이 중 약 1100여개 매장의 매장 정보와 일 매출 데이터를 다루고 있습니다.

2015년 Rossmann에서 직접 Kaggle에 의뢰하여 상금 35000달러 규모의 competition을 개최했습니다. 
기존에는 각 지점 매니저들이 자신만의 기준을 가지고 향후 6주 간의 예상 매출량을 예측해 보고하는 구조였는데, 노력이 많이 들 뿐만 아니라 그 정확성이 들쭉날쭉이라서 보다 정확한 판매량 예측 모델을 만드는 것이 목표입니다.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# loading packages

import numpy as np
import pandas as pd
from pandas import datetime
from pandas import Series,DataFrame

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns # advanced vizs
%matplotlib inline

In [ ]:
# importing train data files
store_df= pd.read_csv('../input/rossmann-store-sales/store.csv')
train_df= pd.read_csv('../input/rossmann-store-sales/train.csv', parse_dates = True, low_memory = False, index_col = 'Date')
test_df = pd.read_csv('../input/rossmann-store-sales/test.csv', parse_dates = True, low_memory = False, index_col = 'Date')
state = pd.read_csv("../input/rossmann-store-extra/store_states.csv")
state_name = pd.read_csv("../input/rossmann-store-extra/state_names.csv")
weathers = pd.read_csv("../input/rossmann-store-extra/weather.csv")

# Getting to Know your Data

본 데이터는 매장별 정보를 다루는 store.csv와 일별 정보를 다룬 train.csv/test.csv가 분리되어 있습니다.
먼저 매장별 정보를 다루는 store.csv 데이터를 살펴보겠습니다.

## 1. store.csv 살펴보기

In [ ]:
store_df.head()

In [ ]:
store_df.info()

### 변수 정리
store type: 매장 종류. a,b,c,d 외에 다른 코멘트 없음  
assortment: 상품 진열, 구색 수준 a = basic, b = extra, c = extended  
CompetitionDistance: 가장 가까운 경쟁 매장과의 거리  
CompetitionOpenSince(Month/Year): 경쟁 매장이 오픈한 시기
Promo2: 일부 매장에서의 지속적이고 연속적인 프로모션  
Promo2Since(Year/Week): 그 매장에서 Promo2를 시작한 시기  
PromoInterval: Promo2 프로모션의 전개 간격. Jan,Apr,Jul,Oct이면, 1월, 4월, 7월, 10월에 프로모션을 진행함


In [ ]:
store_df.describe()

총 1115개 매장에 대한 정보를 담고 있습니다.  
가장 가까운 경쟁 매장과의 거리는 min이 20으로 0인 경우는 없고, max는 무료 75860입니다.  
경쟁 매장의 오픈 시기의 min 1900은 오류가 아닌가 의심이 갑니다.
매장에서 진행하는 연속 프로모션은 mean이 0.5로 딱 절반 정도가 promotion 중입니다.  

In [ ]:
store_df.describe(include ="O")

store type과 assortment level은 a,b,c와 같은 형태로 기입되어 있습니다.
promotion interval 역시 문자열로 기입되어 있는데 1,4,7,10월 / 2,5,8, 11월 / 3,6,9, 12월 의 세 가지 값으로 구성되어 있습니다.

In [ ]:
#Checking the no. of NaN vales
store_df.isna().sum()

store.csv에는 결측치가 상당히 높은 feature들이 있다는 것을 알 수 있습니다.  
CompetitionDistance는 가장 가까운 경쟁 매장과의 거리를 가리키는 변수이고,  
CompetitionOpenSinceMonth, CompetitionOpenSinceYear는 이에 종속된 변수입니다.  
Promo2는 매장별 특별 promotion을 하는 지의 유(1) 무(0) 변수이고,  
Promo2SinceWeek,Promo2SinceYear,PromoInterval는 이에 종속된 변수입니다.


In [ ]:
store_df['Promo2'].value_counts()

In [ ]:
store_df.loc[store_df['Promo2']==0]

In [ ]:
store_df.loc[store_df['Promo2']==0, 'Promo2SinceWeek'].unique()

promo2의 value가 0인 경우에는 관련된 feature인 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval'의 value가 NaN로 입력되어있음을 알 수 있습니다.

## 2. train.csv 살펴보기

In [ ]:
train_df.head()

### 변수 정리

DayOfWeek: 요일. 월요일(1)~일요일(7)순  
StateHoliday: 국가,또는 주에서 지정하는 휴일 주말을 포함하지 않음 a:공휴일 b:부활절 c:크리스마스 0: None  
SchoolHoliday: 학교 쉬는 날. 공휴일은 모두 포함됨

In [ ]:
train_df.info()

train data set에는 결측치가 없었습니다.

In [ ]:
test_df.tail()

test.csv에만 id feature가 있습니다.  
y value인 sales외에도 고객 수인 customer도 없습니다.

In [ ]:
round(train_df.describe(), 2)

In [ ]:
round(test_df.describe(), 2)

train.csv와 test.csv feature들은 대체적으로 비슷한 분포로 나타났지만, school holiday에서는 큰 차이를 보였습니다.  
test.csv에서는 school holiday의 비율이 44%나 되었습니다.
train data set은 3년 정도 긴 기간 동안의 data인 반면, test는 몇 개월의 짧은 기간 동안의 data라 방학 기간이 겹친 게 아닌가 합니다.  
test data의 date: 2015-08-01 ~ 2015-09-17

In [ ]:
train_df['StateHoliday'].value_counts()

In [ ]:
train_df['SchoolHoliday'].value_counts()

In [ ]:
train_df['SchoolHoliday'].groupby(train_df['StateHoliday']).mean()

부활절(b), 크리스마스(c)에는 학교는 거의 다 쉬고, 그 밖의 공휴일에는 그렇지 않을 때보다 약간 더 학교가 쉴 확률이 높은 정도인 것을 알 수 있습니다.

In [ ]:
train_df['DayOfWeek'].value_counts()

연속된 기간 동안의 시계열 데이터라서 요일은 거의 동일하게 분포되어 있습니다.

# Data Cleaning

In [ ]:
temp_train=train_df.copy()
temp_store=store_df.copy()

## 1. 결측치 처리하기

In [ ]:
temp_store.isna().sum()

CompetitionDistance의 경우는 결측치가 매우 적은 연속 변수여서 중간값으로 대체한다.

In [ ]:
temp_store.sort_values(by='CompetitionDistance')

In [ ]:
temp_store.CompetitionDistance.fillna(temp_store.CompetitionDistance.median(),inplace=True)

CompetitionOpenSinceMonth, CompetitionOpenSinceYear 가 NaN인 case는?

In [ ]:
store_df[store_df['CompetitionOpenSinceYear'].isna() == True].describe()

NaN이 아닌 case와 큰 차이가 없는 것을 알수 있고, 연속변수가 아니므로 최빈값으로 대체하겠습니다.

In [ ]:
#temp_store.CompetitionOpenSinceMonth.fillna(temp_store.CompetitionOpenSinceMonth.mode()[0],inplace=True)
#temp_store.CompetitionOpenSinceYear.fillna(temp_store.CompetitionOpenSinceYear.mode()[0],inplace=True)

Promo2SinceYear, Promo2SinceWeek, PromoInterval feature는 결측치가 거의 절반 가까이 됩니다.  
NaN인 case들을 살펴보겠습니다.

In [ ]:
temp_store.query('Promo2SinceYear.isnull()', engine='python')

In [ ]:
temp_store.query('PromoInterval.isnull() & Promo2 == 0', engine='python')

Promo2SinceYear, Promo2SinceWeek, PromoInterval feature 들은 Promo2 value가 0인 row와 동일하다는 것을 알 수 있습니다.  
결측치들은 모두 0으로 대체하겠습니다.

In [ ]:
temp_store.CompetitionOpenSinceMonth.fillna(0, inplace=True)
temp_store.CompetitionOpenSinceYear.fillna(0, inplace=True)
temp_store.Promo2SinceWeek.fillna(0, inplace=True)
temp_store.Promo2SinceYear.fillna(0, inplace=True)
temp_store.PromoInterval.fillna(0, inplace=True)

In [ ]:
temp_store.info()

## 2. 시간 관련 변수 처리하기

In [ ]:
temp_train

'연-월-일' 형태였던 Date feature를 분석하기 좋게 분리하여 저장하겠습니다. 

In [ ]:
temp_train['Year'] = temp_train.index.year
temp_train['Month'] = temp_train.index.month
temp_train['Day'] = temp_train.index.day
temp_train['WeekOfYear'] = temp_train.index.weekofyear
temp_train['Week'] = temp_train.index.week%4+1

## 3. outlier 처리하기

### 1) sales

In [ ]:
plt.figure(figsize=(12,12))
sns.set(style="whitegrid")
sns.boxplot(data=temp_train,x="DayOfWeek",y="Sales")

매출량의 평균은 월요일이 높고, 일요일이 낮은 것을 알 수 있습니다.  
특히 일요일의 경우 3분위 값까지 0인 것을 알 수 있는데, 일요일에는 대부분 매장이 오픈하지 않아 매출량도 0이기 때문입니다.  
그러나 outlier로 찍힌 value들을 보면 다른 요일의 매출액과 거의 차이가 없는 수치를 보이는데, 문을 연 경우에는 높은 매출을 보인 것을 알 수 있습니다.

In [ ]:
sns.countplot(x = 'DayOfWeek', hue = 'Open', data = temp_train)
plt.title('Store Daily Open Countplot')

In [ ]:

sns.set(style="whitegrid")
sns.histplot(data=temp_train,x="Sales")

customer와 sales가 0인 경우가 매우 많고, 그 외에는 대체로 정규분포를 따른다.
sales 0 은 대부분 매장이 문을 닫았을 때일 것 같은데, 문을 열었을 때만 보면 어떨까?

In [ ]:
sns.displot(data=temp_train,x="Sales", kind="ecdf")

경험적 누적분포 함수로 보면 sales가 0인 경우가 거의 20%, 10000유로 이하가 90% 가까이 되는데 최대값은 40000이 넘는다.  
scaling필요

매장이 닫은 날에는 매출이 0이라서 그럴까?

In [ ]:
open_df = temp_train.loc[temp_train['Open']==1]
open_df.loc[open_df['Sales']==0].shape

In [ ]:
sns.set(style="whitegrid")
sns.histplot(data=open_df,x="Sales")

In [ ]:
sns.displot(data=open_df,x="Sales", kind="ecdf")

매장이 연 날에도 매출이 0인 날이 있기는 하지만, 매우 미미하다.
오류인 것일까?  
매장 문을 닫은 날은 drop하는 것이 더 modeling 결과가 좋게 나올까?

In [ ]:
temp_train2 = temp_train[(temp_train["Open"] != 0) & (temp_train['Sales'] != 0)]

print(temp_train2.shape)

In [ ]:
temp_train2.shape[0]/temp_train.shape[0]

전체 case중에 약 83%, 총 84만 여 row가 남는다.

### 2. competition since year

경쟁매장의 출점 연도는 대부분 1990년에서 2014년 사이의 값이지만, 1900년과 1961년에 각 1건 씩 있습니다. error인지 의심스러운 값입니다.

In [ ]:
temp_store.sort_values(by='CompetitionOpenSinceYear')

In [ ]:
temp_store.CompetitionOpenSinceYear.replace(1900, int(temp_store.CompetitionOpenSinceYear.mode()[0]), inplace=True)
temp_store.CompetitionOpenSinceYear.replace(1961, int(temp_store.CompetitionOpenSinceYear.mode()[0]), inplace=True)

In [ ]:
temp_store.sort_values(by='CompetitionOpenSinceYear')

## 6. categorical 변수 처리하기

In [ ]:
temp_train['StateHoliday'].unique()

In [ ]:
data=temp_train.loc[temp_train['StateHoliday']!='0']

In [ ]:
sns.violinplot(data=data,x='StateHoliday',y='Sales',hue='Promo', split = True)

휴일 a,b,c 분포에 유의미한 차이가 없으므로, 휴일인지(1), 아닌지(0)로 변환합니다.

In [ ]:
temp_train["StateHoliday"]=np.where(temp_train["StateHoliday"] == '0' ,0,1)

In [ ]:
temp_train['StateHoliday'].value_counts()

In [ ]:
temp_train.groupby(['StateHoliday']).mean()

store data의 store type과 assortment feature를 살펴봅시다.
store type column의 a, b, c, d 의 value가 있는데, data 문서에서 그 내용이나 차이에 대한 언급이 없습니다.

Assortment 변수도 a = basic, b = extra, c = extended 이므로 numerical 변수로 변환

In [ ]:
temp_store['Assortment'] =[1 if i == 'a' else 2 if i == 'b' else 3 for i in temp_store['Assortment']]

In [ ]:
temp_store['Assortment'].value_counts()

# 연속형 변수 처리하기

In [ ]:
# adding new variable
temp_train['SalePerCustomer'] = temp_train['Sales']/temp_train['Customers']
temp_train['SalePerCustomer'].describe()

고객 한명 당 매출액 변수를 만들어 보았습니다.
평균적으로 9.5 유로 정도를 소비하며 75분위가 11유로인데 최대값이 64유로로 outlier임
log 하는 것이 좋을까?

# DataFrame 합치기

In [ ]:
#가게 문 닫고 매출도 안나온 날은 빼기
temp_train2 = temp_train[(temp_train["Open"] != 0) & (temp_train['Sales'] != 0)]

In [ ]:
train_final=temp_store.merge(temp_train2,on=["Store"],how="inner")
train_final.head()

경쟁 매장의 오픈시기, 프로모션의 시작시기에 관련된 변수들을 분석하기 좋은 형태로 변환하겠습니다.

In [ ]:
train_final['CompetitionOpen'] = 0
train_final['CompetitionOpen'] = train_final['CompetitionOpen'].where(train_final['CompetitionOpenSinceYear'] == 0, other=12 * (train_final['Year'] - train_final['CompetitionOpenSinceYear']) + (train_final['Month'] - train_final['CompetitionOpenSinceMonth']))
train_final['PromoOpen'] = 0
train_final['PromoOpen'] = train_final['PromoOpen'].where(train_final['Promo2SinceYear'] == 0, other=12 * (train_final['Year'] - train_final['Promo2SinceYear']) + (train_final['WeekOfYear'] - train_final['Promo2SinceWeek'])/4)

train_final

In [ ]:
train_final['CompetitionOpen'] = train_final['CompetitionOpen'].where(train_final['CompetitionOpen'] > 0,  0)
train_final['PromoOpen'] = train_final['PromoOpen'].where(train_final['PromoOpen'] > 0,  0)


In [ ]:
sns.lineplot(x='PromoOpen', y='Sales', data=train_final)

In [ ]:
sns.lineplot(x='CompetitionOpen', y='Sales', data=train_final)

# EDA

## promotion이 매출량을 늘릴까?

In [ ]:
plt.figure(figsize=(10,6))
sns.set(style="whitegrid")
sns.violinplot(x='DayOfWeek',y='Sales',hue='Promo',split=True,data=train_final)

promotion이 있을 때 매출액 분포가 조금 더 높은 것을 알 수 있다.  
토요일과 일요일에는 promotion을 하지 않는다는 것을 알 수 있다.  
일요일에는 문을 닫는 점포가 많아서 매출액 0이 많지만, 문을 열고 매출액이 발생하는 case도 있으며, 매출액의 분포도 매우 넖었다.

## 경쟁점포와의 거리가 매출액과 관련이 있을까?

In [ ]:
plt.figure(figsize=(12,6))
sns.set(style="whitegrid")
sns.lmplot(x='CompetitionDistance',y='Sales',data=train_final)

크게 상관관계가 없어 보인다.

## 휴일과 매출액은 상관있을까?

In [ ]:
plt.figure(figsize=(10,6))
sns.set(style="whitegrid")
sns.violinplot(x='StateHoliday',y='Sales',hue='Promo',split=True,data=train_final)

StateHoliday ==1 즉 공휴일에는 매출액의 중간값이 매우 낮았다. 문을 닫은 날이 많아서 그런 것 같다.

In [ ]:
plt.figure(figsize=(10,6))
sns.set(style="whitegrid")
sns.violinplot(x='SchoolHoliday',y='Sales',hue='Promo',split=True,data=train_final)

반면, school holiday인 날에는 그렇지 않은 날과 비슷한 분포를 보였다.

## Store type에 따른 판매 양상의 차이가 있을까?

In [ ]:
train_final.groupby('StoreType').size()

In [ ]:
sns.countplot(x='StoreType', data=train_final,order=['a','b','c','d'])

In [ ]:
train_final.groupby('StoreType')['Sales'].describe()


'b'타입의 매장은 매장수는 현저히 적은 반면, 매출액은 현저히 높았습니다. 평균값 뿐 아니라 1분위, 3분위 값도 높아, 다른 타입의 매장보다 매출액이 더 높다고 말할 수 있었습니다.

In [ ]:
plt.figure(figsize=(10,8))
sns.boxplot(x='StoreType', y='Sales', data=train_final, order=['a','b','c','d'])

In [ ]:
plt.figure(figsize=(10,8))
sns.boxplot(x='StoreType', y='SalePerCustomer', data=train_final, order=['a','b','c','d'])

전체 매출액이 가장 높았던 b 타입의 매장이 고객당 매출액은 가장 낮았습니다.
b타입의 매장은 많은 고객들이 소액의 물건을 사가는 형태의 매장이라는 유추가 가능합니다.
한편, d타입의 매장이 고객당 매출액은 가장 높았습니다.
store type 변수는 매장 별로 특징이 있다는 점은 발견했지만, 순서가 있는 labeling이 가능한 feature는 아닌 것 같습니다.

## 시기에 따른 매출액의 차이가 있을까?


In [ ]:
plt.figure(figsize=(10,6))
sns.set(style="whitegrid")
sns.lineplot(x='Date',y='Sales',data=temp_train)

In [ ]:
train_final['Season']=np.where(train_final['Month'].isin([3,4,5]), "Spring"
                    , np.where(train_final['Month'].isin([6,7,8]), "Summer"
                    , np.where(train_final['Month'].isin([9,10,11]), "Fall"
                    , np.where(train_final['Month'].isin([12,1,2]), "Winter", "None"))))

In [ ]:
sns.boxplot(x='Season', y='Sales', data=train_final, order=['Spring','Summer','Fall','Winter'])

계절에 따른 차이는 매출액의 차이는 거의 없는 것으로 보인다.  
월에 따른 차이는 있을까?  
년에 따른 차이는?

In [ ]:
sns.lineplot(data = train_final, x = 'Year', y = "Sales")

In [ ]:
sns.lineplot(data = train_final, x = 'Month', y = "Sales")

In [ ]:
sns.lineplot(data = train_final, x = 'Week', y = "Sales")

In [ ]:
sns.lineplot(data = train_final, x = 'Day', y = "Sales")

In [ ]:
sns.factorplot(data = train_final, x = 'Month', y = "Sales", 
               col = 'StoreType',
               hue = 'StoreType',
               row = 'Promo',
               col_order=['a', 'b', 'c', 'd']
               ) 

11월과 12월에 매출액이 상승하는 패턴이 보인다.
특히 프로모션이 있는 경우 매출액 상승이 더 커보인다.

해에 따른 매출액의 유의미한 패턴은 보이지 않는다.

In [ ]:
del train_final['Season']

### 고객당 구매액과 매출액에 상관관계가 있을까?

In [ ]:
plt.figure(figsize=(10,8))
sns.scatterplot(x='SalePerCustomer', y='Sales',hue= 'Promo', data=train_final, alpha=0.5)

분석에 필요하지 않은 변수들을 삭제합니다.

In [ ]:
train_final.drop(columns=['Store','CompetitionOpenSinceMonth','CompetitionOpenSinceYear','Promo2SinceWeek','Promo2SinceYear','WeekOfYear'], inplace=True)

In [ ]:
train_final.info()

# feature engineering

## 1) OneHot Encoding
범주형 변수인 store type과 promo2interval 을 one hot encoding으로 변환합니다. 

In [ ]:
df2 = pd.get_dummies(train_final['StoreType'], drop_first=True)
df2

In [ ]:
df3= pd.get_dummies(train_final['PromoInterval'], drop_first=True)
df3

In [ ]:
df4 = pd.concat([df2,df3],axis=1)
df4

In [ ]:
train_final

In [ ]:
train_f2 = pd.concat([df4,train_final],axis=1)


## 2) scaling

분포가 왼쪽으로 기울어져있고, 최대값이 너무 높은 형태의 feature 들을 log값을 취해 변환합니다.  
sales  
customers  
competition distance  

In [ ]:
sns.histplot(data=train_f2,x="Sales")

In [ ]:
 train_f2['ln_Sales'] = train_f2['Sales'].map(lambda x : np.log(x) if x != 0 else 0)

In [ ]:
sns.histplot(data=train_f2,x="ln_Sales")

In [ ]:
sns.histplot(data=train_f2,x="CompetitionDistance")

In [ ]:
 train_f2['ln_CompetitionDistance'] = train_f2['CompetitionDistance'].map(lambda x : np.log(x) if x != 0 else 0)

In [ ]:
sns.histplot(data=train_f2,x="ln_CompetitionDistance")

In [ ]:
sns.histplot(data=train_f2,x="Customers")

In [ ]:
train_f2['ln_Customers'] = train_f2['Customers'].map(lambda x : np.log(x) if x != 0 else 0)
sns.histplot(data=train_f2,x='ln_Customers')

In [ ]:
sns.histplot(data=train_f2,x='SalePerCustomer')

0값이 많은 연속형 변수들은 robust scaler로 스케일링합니다.

In [ ]:
from sklearn.preprocessing import RobustScaler
roscaler= RobustScaler()

In [ ]:
train_f2['scaled_PromoOpen'] = roscaler.fit_transform(train_f2[['PromoOpen']])
train_f2['scaled_CompetitionOpen'] = roscaler.fit_transform(train_f2[['CompetitionOpen']])

In [ ]:
train_f2.head()

In [ ]:
train_f2.drop(columns=['PromoOpen','CompetitionOpen'], inplace = True)

In [ ]:
train_f2.drop(columns=['CompetitionDistance','Sales','Customers'], inplace = True)

In [ ]:
train_f2.info()

나머지 연속형 변수들을 Standard Scaling 하겠습니다.

In [ ]:
from sklearn.preprocessing import StandardScaler
std=StandardScaler()

In [ ]:
train_f2['std_SalePerCustomer']=std.fit_transform(train_f2[['SalePerCustomer']])
train_f2['std_ln_CompetitionDistance']=std.fit_transform(train_f2[['ln_CompetitionDistance']])
train_f2['std_ln_Customers']=std.fit_transform(train_f2[['ln_Customers']])
train_f2['std_ln_Sales']=std.fit_transform(train_f2[['ln_Sales']])

In [ ]:
train_f2.drop(columns=['ln_Customers','SalePerCustomer','ln_CompetitionDistance','ln_Sales','std_ln_Customers'], inplace = True)

In [ ]:
train_f2.info()

In [ ]:
train_f2.drop(columns=['PromoInterval','StoreType'], inplace = True)

# ML

x와 y를 분리합니다.

In [ ]:
x=train_f2.drop(['std_ln_Sales'], axis=1)

In [ ]:
y=train_f2['std_ln_Sales']

In [ ]:
x.info()

In [ ]:
y.head()

train, test set을 분리합니다.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

## 1) Multiple Linear Regression model

In [ ]:
from sklearn.linear_model import LinearRegression #모델 불러오기

In [ ]:
# 모델 정의하기 = 인스턴스화= 객체화
m_lr = LinearRegression()
# 학습하기
m_lr.fit(x_train, y_train)

In [ ]:
#결과 예측하기
y_pred = m_lr.predict(x_test)

In [ ]:
#결과 살펴보기
from sklearn.metrics import r2_score as r2, mean_squared_error as mse
import math

In [ ]:
#설명력
print('R^2: ', r2(y_test,y_pred))

# RMSE 예측력 : 해석을 위해서 
print('RSME: ',math.sqrt(mse(y_test,y_pred)))

In [ ]:
#계수와 절편을 프린트해라
print(m_lr.intercept_)
print(m_lr.coef_)

In [ ]:
c1=m_lr.coef_.reshape(1,-1)
m_lr.coef_.shape
c2=pd.DataFrame(c1, columns=list(x_test.columns))

In [ ]:
c2.T

In [ ]:
plt.figure(figsize=(20,6))
sns.barplot(data=c2)